# COMPUSTAT - Data Extraction

## Import Libraries

In [1]:
# import necessary libraries
import wrds
import pandas as pd

# note: wrds provides an api to wharton research data services which provides access to various databases
# note: compustat can be accessed through wrds

# connect to wrds
db = wrds.Connection(wrds_username='acka662')

Loading library list...
Done


## NYSE Ticker Symbols

In [2]:
# note: a collection of ticker symbols must be provided in order to search compustat;
# note: this study is based on companies listed on the nyse;
# note: a csv file containing the ticker symbols of the nyse has been obtained from the nasdaq website

# load the NYSE_ticker_list.csv file into a pandas dataframe
NYSE_df = pd.read_csv('NYSE_ticker_list.csv')
# display NYSE_df
display(NYSE_df.head())
# row count of NYSE_df
print('NYSE_df row count:', NYSE_df.shape[0])
print(NYSE_df.info())

,Ticker_Symbol,Company_Name,Last_Sale,Market_Capitalization,ADR_TSO,IPO_Year,Sector,Industry,Summary_Quote
0,DDD,3D Systems Corporation,7.00,8.270659e+08,NaN,NaN,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
1,MMM,3M Company,161.72,9.303413e+10,NaN,NaN,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
2,WBAI,500.com Limited,9.25,3.975212e+08,NaN,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://www.nasdaq.com/symbol/wbai
3,WUBA,58.com Inc.,53.79,7.997253e+09,NaN,2013.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,EGHT,8x8 Inc,24.31,2.413679e+09,NaN,NaN,Technology,EDP Services,https://www.nasdaq.com/symbol/eght


NYSE_df row count: 3123
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3123 entries, 0 to 3122
Data columns (total 9 columns):
Ticker_Symbol            3123 non-null object
Company_Name             3123 non-null object
Last_Sale                3021 non-null float64
Market_Capitalization    3123 non-null float64
ADR_TSO                  0 non-null float64
IPO_Year                 1457 non-null float64
Sector                   2117 non-null object
Industry                 2117 non-null object
Summary_Quote            3123 non-null object
dtypes: float64(4), object(5)
memory usage: 219.7+ KB
None


In [3]:
# remove unnecessary columns from NYSE_df
NYSE_df = NYSE_df.drop(columns=['Last_Sale', 'Market_Capitalization', 'ADR_TSO', 'IPO_Year', 'Summary_Quote'], axis=1)

# count missing values in each column of NYSE_df
for col in NYSE_df:
    print('Missing values in %s: %d' % (col, NYSE_df[col].isnull().sum().sum()))
# remove any rows in NYSE_df containing missing values and reset index
NYSE_df = NYSE_df.dropna().reset_index(drop=True)

# note: finance companies are excluded from this study
# remove any rows in NYSE_df belonging to the finance sector
fin_idx = NYSE_df[NYSE_df['Sector']=='Finance'].index
NYSE_df = NYSE_df.drop(fin_idx, axis=0).reset_index(drop=True)

# remove any spaces in Symbol column of NYSE_df
NYSE_df['Sector'] = NYSE_df['Sector'].str.strip()

Missing values in Ticker_Symbol: 0
Missing values in Company_Name: 0
Missing values in Sector: 1006
Missing values in Industry: 1006


## Data Extraction & Wrangling

In [4]:
# note: the following variables are to be retrieved from compustat for the ticker symbols in NYSE_df....
# - central index key (cik)
# - ticker symbol (tic)
# - company name (conm)
# - data date (datadate)
# - market capitalization (mkvalt)
# - total assets (at)
# - total liabilities (lt)
# - income before extraordinary items (ibadj)
# - cash flow from operations (oancf)
# - total long-term debt (dltt)
# - current assets (act)
# - current liabilities (lct)
# - common shares outstanding (csho)
# - total sales (sale)
# - cost of goods sold (cogs)
# note: the variables above need to be retrieved for each fiscal year from 01/01/2009 to 01/08/2019
# note: the accounting variables have units equal to $ millions (i.e. $8,000,000 = 8)

# store ticker symbols from NYSE_df as dictionary
tic_sym = {'ticker_symbol':tuple(NYSE_df['Ticker_Symbol'].values.T.tolist())}
# query fundamentals annual table of compustat to retrieve variables and store in dataframe
# note: data is stored in a postgresql database management system
funda_df = db.raw_sql("""SELECT DISTINCT cik, tic, conm, datadate, mkvalt, at, lt,
                        ibadj, oancf, dltt, act, lct, csho, sale, cogs
                        FROM comp.funda WHERE tic IN %(ticker_symbol)s
                        AND datadate>='01/01/2009'
                        AND datadate<='08/01/2019'
                        AND cik IS NOT NULL
                        AND mkvalt IS NOT NULL
                        AND at IS NOT NULL
                        AND lt IS NOT NULL
                        AND ibadj IS NOT NULL
                        AND oancf IS NOT NULL
                        AND dltt IS NOT NULL
                        AND act IS NOT NULL
                        AND lct IS NOT NULL
                        AND csho IS NOT NULL
                        AND sale IS NOT NULL
                        AND cogs IS NOT NULL""",
                    params=tic_sym)
# display funda_df
display(funda_df.head())
# row count of funda_df
print('funda_df row count:', funda_df.shape[0])

,cik,tic,conm,datadate,mkvalt,at,lt,ibadj,oancf,dltt,act,lct,csho,sale,cogs
0,0000001750,AIR,AAR CORP,2009-05-31,571.5948,1377.511,720.616,80.600,64.451,392.984,851.312,254.418,38.884,1423.976,1110.677
1,0000001750,AIR,AAR CORP,2010-05-31,777.8348,1501.042,754.692,44.628,153.156,336.191,863.429,325.550,39.484,1352.151,1065.902
2,0000001750,AIR,AAR CORP,2011-05-31,1049.8206,1703.727,868.438,70.860,108.598,329.802,913.985,416.010,39.781,1775.782,1408.071
3,0000001750,AIR,AAR CORP,2012-05-31,485.2897,2195.653,1329.631,65.178,94.217,669.489,1063.272,473.226,40.273,2074.498,1662.408
4,0000001750,AIR,AAR CORP,2013-05-31,790.0029,2136.900,1217.400,53.000,162.900,622.200,1033.700,389.000,39.382,2167.100,1714.500


funda_df row count: 10109


In [5]:
# rename columns in funda_df
funda_df = funda_df.rename(columns={'cik':'CIK',
                                    'tic':'Ticker_Symbol',
                                    'conm':'Company_Name',
                                    'datadate':'Data_Date',
                                    'mkvalt':'Market_Capitalization',
                                    'at':'Total_Assets',
                                    'lt':'Total_Liabilities',
                                    'ibadj':'Net_Income_Before_Extra_Items',
                                    'oancf':'Cash_Flow_From_Operations',
                                    'dltt':'Total_Long_Term_Debt',
                                    'act':'Current_Assets',
                                    'lct':'Current_Liabilities',
                                    'csho':'Common_Shares_Outstanding',
                                    'sale':'Total_Sales',
                                    'cogs':'Cost_Of_Goods_Sold'})
# store the sector for each company in a new column in funda_df
tic_sec_dict = pd.Series(NYSE_df['Sector'].values, index=NYSE_df['Ticker_Symbol']).to_dict()
funda_df['Sector'] = funda_df['Ticker_Symbol'].map(tic_sec_dict)

# count missing values in each column of funda_df
for col in funda_df:
    print('Missing values in %s: %d' % (col, funda_df[col].isnull().sum().sum()))

Missing values in CIK: 0
Missing values in Ticker_Symbol: 0
Missing values in Company_Name: 0
Missing values in Data_Date: 0
Missing values in Market_Capitalization: 0
Missing values in Total_Assets: 0
Missing values in Total_Liabilities: 0
Missing values in Net_Income_Before_Extra_Items: 0
Missing values in Cash_Flow_From_Operations: 0
Missing values in Total_Long_Term_Debt: 0
Missing values in Current_Assets: 0
Missing values in Current_Liabilities: 0
Missing values in Common_Shares_Outstanding: 0
Missing values in Total_Sales: 0
Missing values in Cost_Of_Goods_Sold: 0
Missing values in Sector: 0


In [6]:
# funda_df information
funda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10109 entries, 0 to 10108
Data columns (total 16 columns):
CIK                              10109 non-null object
Ticker_Symbol                    10109 non-null object
Company_Name                     10109 non-null object
Data_Date                        10109 non-null object
Market_Capitalization            10109 non-null float64
Total_Assets                     10109 non-null float64
Total_Liabilities                10109 non-null float64
Net_Income_Before_Extra_Items    10109 non-null float64
Cash_Flow_From_Operations        10109 non-null float64
Total_Long_Term_Debt             10109 non-null float64
Current_Assets                   10109 non-null float64
Current_Liabilities              10109 non-null float64
Common_Shares_Outstanding        10109 non-null float64
Total_Sales                      10109 non-null float64
Cost_Of_Goods_Sold               10109 non-null float64
Sector                           10109 non-null object


In [7]:
# rearrange column order of funda_df
funda_cols = funda_df.columns.to_list()
funda_cols = funda_cols[0:3] + funda_cols[-1:] + funda_cols[3:-1]
funda_df = funda_df[funda_cols]

# note: public listed companies can have more than one 'class' of stock
# note: the different classes of a stock tend to have different ticker symbols but identical cik numbers
# note: therefore, it is necessary to check for any cik duplicates

# check for duplicate cik values with distinct ticker symbols
print('Number of unique cik values:', len(set(funda_df['CIK'])))
print('Number of unique ticker symbols:', len(set(funda_df['Ticker_Symbol'])))
print('\n')
if len(set(funda_df['CIK']))==len(set(funda_df['Ticker_Symbol'])):
    print('There are no duplicate values!')
else:
    print('There are duplicate values! These need to be removed!')

Number of unique cik values: 1237
Number of unique ticker symbols: 1237


There are no duplicate values!


## Save Data to CSV File

In [8]:
# save funda_df to csv file
funda_df.to_csv('fundamentals.csv', index=False)

In [9]:
# close connection to wrds
db.close()